# Description
I got all the data and timestamp values\
To do this i checked that every element had unique `top` values in the style attribute so i used this as a criteria\
so as to not visit the element twice.\
If visited but not got the timestamp value then the element can be visited again.\
I scrolled the page by small values rather than large values so as to not skip some values\

In [2]:
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import numpy as np
import time

pd.set_option('display.max_rows', None)

In [134]:
url = "https://opensea.io/nickname808/activity?search[eventTypes][0]=OFFER_ENTERED&search[eventTypes][1]=AUCTION_CREATED"

In [14]:
#get the top value of elements
def get_top(style):
    idx = style.index("top")
    top = ""
    for px in style[idx+5: ]:
        if px == ";":
            break
        top += px
    return top

In [ ]:
from_t = 246
visited = []
err = []
data_str = []
driver = webdriver.Chrome()   #generate a driver
driver.get(url)               #going to the url
driver.maximize_window()     #maximizes the window
time.sleep(3)                #waits 2 seconds after loading
screen_height = driver.execute_script("return window.screen.height;") #get current screen height
i = 0
sol = ""
#This loop will run until we reach the end of the infinite scroll
action = ActionChains(driver)
while True:
    #scroll the page to the next screen height one by one
    driver.execute_script("window.scrollTo(0, {screen_height}+{i});".format(screen_height=screen_height, i=i))
    i += 300
    time.sleep(3)
    #get all the text values
    divs = driver.find_elements(By.TAG_NAME, "div")
    time.sleep(3)
    for div in divs:
        try:
            s = ""
            if div.get_attribute("role") == "listitem":
                top = get_top(div.get_attribute("style"))
                if top not in visited:
                    visited.append(top)
                    if div.text != '':
                        s += div.text 
                        s += "\n"
                        
                    spans = div.find_elements(By.TAG_NAME, "span")
                    time.sleep(3)
                    for span in spans:
                        if span.get_attribute("aria-expanded") == "false":
                            action.move_to_element(span).perform()
                            time.sleep(4)
                            try:
                                stamp = driver.find_element(By.CLASS_NAME, "tippy-content").text
                                s += stamp
                            except:
                                s += "NULL"
                                err.append(top)

                    print("---------------------")
                    if "NULL" not in s:
                        if s not in data_str:
                            data_str.append(s)
                    else:
                        visited.remove(top)
        except:
            continue
    time.sleep(3)
    #get current scrollHeight
    scroll_height = driver.execute_script("return document.body.scrollHeight;")
    print(screen_height + i, scroll_height)
    #break the loop if the screen height exceeds the scroll height
    if (screen_height) + i > scroll_height:
        break
driver.close()       

In [136]:
len(data_str)

146

In [151]:
# converting strings to list 
for d_s in data_str:
    data_all.append(d_s.split("\n"))

In [171]:
# removing unwanted values
for d_a in data_all:
    if 'sell' in d_a:
        d_a.remove('sell')

In [176]:
status = ['Accepted', 'Canceled', 'Expired', 'Sold']

In [180]:
# filling values that are not available as information
for d_a in data_all:
    if d_a[1] not in status:
        d_a.insert(1, 'Not Available')
        
for d_a in data_all:
    if d_a[6] == "--":
        d_a[6] = "Not Available"
    if d_a[9] == "---":
        d_a[9] = "Not Available"

In [211]:
data_all[-2:]

[['List',
  'Not Available',
  'PunksUnchained #7730',
  'Punks Unchained (ETH)',
  '0.015 ETH',
  '$26.46',
  'Not Available',
  '1',
  'nickname808',
  'Not Available',
  '1y ago',
  'April 15, 2022 at 12:09 PM'],
 ['List',
  'Not Available',
  'Stone',
  'Sunflower Land Collectibles',
  '0.0017 ETH',
  '$2.91',
  'Not Available',
  '2',
  'nickname808',
  'Not Available',
  '11mo ago',
  'April 16, 2022 at 12:50 AM']]

In [197]:
df = pd.DataFrame(data_all, columns=["Event Type", "Status", "NFT", "Collection", "Price", "Price (in dollar)", "Rarity", "Quantity", "From", "To", "Time", "Timestamp", "IDK"])

In [202]:
df.drop("IDK", axis=1, inplace=True)

In [255]:
df.shape

(146, 12)

In [204]:
df.head()

,Event Type,Status,NFT,Collection,Price,Price (in dollar),Rarity,Quantity,From,To,Time,Timestamp
0,Offer,Canceled,Lunar Calendar,Sunflower Land Collectibles,4.700 USDC,$4.73,Not Available,10,nickname808,Not Available,4d ago,"March 22, 2023 at 2:51 PM"
1,Offer,Accepted,Tiki Totem,Sunflower Land Collectibles,4.600 USDC,$4.63,Not Available,6,nickname808,Not Available,6d ago,"March 20, 2023 at 10:19 PM"
2,Offer,Canceled,Tiki Totem,Sunflower Land Collectibles,4.500 USDC,$4.53,Not Available,10,nickname808,Not Available,6d ago,"March 20, 2023 at 6:16 PM"
3,Offer,Accepted,Lunar Calendar,Sunflower Land Collectibles,5 USDC,$5.03,Not Available,10,nickname808,Not Available,6d ago,"March 20, 2023 at 5:43 PM"
4,Offer,Canceled,Carrot Sword,Sunflower Land Collectibles,115 USDC,$115.69,Not Available,2,nickname808,Not Available,6d ago,"March 20, 2023 at 5:21 PM"


In [208]:
df.isnull().sum()

Event Type           0
Status               0
NFT                  0
Collection           0
Price                0
Price (in dollar)    0
Rarity               0
Quantity             0
From                 0
To                   0
Time                 0
Timestamp            0
dtype: int64

In [288]:
df.head()

,Event Type,Status,NFT,Collection,Price,Price (in dollar),Rarity,Quantity,From,To,Time,Timestamp
0,Offer,Canceled,Lunar Calendar,Sunflower Land Collectibles,4.700 USDC,$4.73,Not Available,10.0,nickname808,Not Available,4d ago,"March 22, 2023 at 2:51 PM"
1,Offer,Accepted,Tiki Totem,Sunflower Land Collectibles,4.600 USDC,$4.63,Not Available,6.0,nickname808,Not Available,6d ago,"March 20, 2023 at 10:19 PM"
2,Offer,Canceled,Tiki Totem,Sunflower Land Collectibles,4.500 USDC,$4.53,Not Available,10.0,nickname808,Not Available,6d ago,"March 20, 2023 at 6:16 PM"
3,Offer,Accepted,Lunar Calendar,Sunflower Land Collectibles,5 USDC,$5.03,Not Available,10.0,nickname808,Not Available,6d ago,"March 20, 2023 at 5:43 PM"
4,Offer,Canceled,Carrot Sword,Sunflower Land Collectibles,115 USDC,$115.69,Not Available,2.0,nickname808,Not Available,6d ago,"March 20, 2023 at 5:21 PM"


In [289]:
# sorting the dataframe according to time of activity
df["Timestamp"] = pd.to_datetime(df["Timestamp"])
df.sort_values("Timestamp", inplace=True, ascending=False)

In [292]:
df.head()

,Event Type,Status,NFT,Collection,Price,Price (in dollar),Rarity,Quantity,From,To,Time,Timestamp
10,Offer,Accepted,Lunar Calendar,Sunflower Land Collectibles,4.700 USDC,$4.73,Not Available,4.0,nickname808,Not Available,12h ago,2023-03-26 00:47:00
8,Offer,Not Available,Lunar Calendar,Sunflower Land Collectibles,4.500 USDC,$4.53,Not Available,10.0,nickname808,Not Available,2d ago,2023-03-24 02:26:00
9,Offer,Accepted,Lunar Calendar,Sunflower Land Collectibles,4.700 USDC,$4.73,Not Available,5.0,nickname808,Not Available,2d ago,2023-03-24 02:21:00
0,Offer,Canceled,Lunar Calendar,Sunflower Land Collectibles,4.700 USDC,$4.73,Not Available,10.0,nickname808,Not Available,4d ago,2023-03-22 14:51:00
15,Offer,Canceled,Lunar Calendar,Sunflower Land Collectibles,4.200 USDC,$4.23,Not Available,10.0,nickname808,Not Available,5d ago,2023-03-21 14:29:00


In [293]:
df["Timestamp"] = df["Timestamp"].dt.strftime("%B %d, %Y at %I:%M %p")

In [295]:
df.head()

,Event Type,Status,NFT,Collection,Price,Price (in dollar),Rarity,Quantity,From,To,Time,Timestamp
10,Offer,Accepted,Lunar Calendar,Sunflower Land Collectibles,4.700 USDC,$4.73,Not Available,4.0,nickname808,Not Available,12h ago,"March 26, 2023 at 12:47 AM"
8,Offer,Not Available,Lunar Calendar,Sunflower Land Collectibles,4.500 USDC,$4.53,Not Available,10.0,nickname808,Not Available,2d ago,"March 24, 2023 at 02:26 AM"
9,Offer,Accepted,Lunar Calendar,Sunflower Land Collectibles,4.700 USDC,$4.73,Not Available,5.0,nickname808,Not Available,2d ago,"March 24, 2023 at 02:21 AM"
0,Offer,Canceled,Lunar Calendar,Sunflower Land Collectibles,4.700 USDC,$4.73,Not Available,10.0,nickname808,Not Available,4d ago,"March 22, 2023 at 02:51 PM"
15,Offer,Canceled,Lunar Calendar,Sunflower Land Collectibles,4.200 USDC,$4.23,Not Available,10.0,nickname808,Not Available,5d ago,"March 21, 2023 at 02:29 PM"


In [ ]:
df.to_csv("__nickname808.csv")
df.to_excel("__nickname808.xls")